In [3]:
%%capture

!jupyter nbconvert --to html Document_Classification.ipynb

In [ ]:
# Carga de las librerías
from google.colab import drive
import contractions
import pandas as pd
import numpy as np
import json
from pandas import json_normalize
from os.path import exists
import os
import re
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, Activation, Softmax
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.initializers import glorot_uniform

In [ ]:
# Conexión a Google Drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#Carga de los datos de entrenamiento
def create_df_from_json(file_dir):
  # returns a dataframe from a json with the following structure:
  # first line is the number of example
  # the reaming of the document contains one json per line
  # parameters
  # file_dir: the absolute route to the file

  df = pd.DataFrame()
  with open(file_dir) as f:
      n = f.readline()
      for i in range(int(n)):
        json_str = f.readline()
        json_in = json.loads(json_str)
        df_tmp = json_normalize(json_in) 
        df = pd.concat([df, df_tmp]) 
  return df

In [ ]:
def load_data(reload, file_out_dir, training_abs_dir, test_in_abs_dir, test_out_abs_dir):
  # params
  # reload: if true, the data is reload from the file sources
  # file_out_dir: directory where to save the merged data from the training and test files
  # training_abs_dir: absolute route to the training file dataset
  # test_in_abs_dir: absolute route to the test file in dataset. It does not include the output
  # test_itest_out_abs_dirn_abs_dir: absolute route to the test output file dataset.
  if not reload and os.path.exists(file_out_dir):
    df = pd.read_csv(file_out_dir)
  else:
    if not os.path.exists(file_out_dir):
      os.mkdir(file_out_dir)
    df_training = create_df_from_json(training_abs_dir)
    df_test = create_df_from_json(test_in_abs_dir)
    df_test = df_test.reset_index(drop=True)
    df_test_out = pd.read_csv(test_out_abs_dir, header=None)
    df_test['category'] = df_test_out
    df = pd.concat([df_test, df_training])
    df =  df.reset_index(drop=True)
    df.to_csv(file_out_dir, index=False)
  return df


In [ ]:
reload = False
file_out_dir = '/content/gdrive/MyDrive/ColabNotebooks/PostClassifier/merged_data.csv'
training_abs_dir = '/content/gdrive/MyDrive/ColabNotebooks/PostClassifier/training.json'
test_in_abs_dir = '/content/gdrive/MyDrive/ColabNotebooks/PostClassifier/sample-test.in.json'
test_out_abs_dir = '/content/gdrive/MyDrive/ColabNotebooks/PostClassifier/sample-test.out.json'
df = load_data(reload, file_out_dir, training_abs_dir, test_in_abs_dir, test_out_abs_dir)



In [ ]:
df.head(15)

,city,section,heading,category
0,chicago,for-sale,Madden NFL 25 XBOX 360. Brand New!,video-games
1,paris.en,housing,looking for room to rent.,wanted-housing
2,newyork,for-sale,two DS game,video-games
3,seattle,housing,map,shared
4,singapore,services,Good Looking Asian Sensation N aughty Girl ---...,therapeutic
5,newyork,for-sale,map,video-games
6,singapore,for-sale,"Panasonic ,2doors fridge(238L)($220 with deliv...",appliances
7,dubai.en,services,SWEET FRIENDLY COMPANION:- monaliza,therapeutic
8,london,services,&#10031; Indulge yourself to the body to body ...,therapeutic
9,chicago,community,Enough,artists


## Data Cleaning

In [ ]:
def remove_contraction(heading):
  expanded_words = []   
  for word in heading.split():
    expanded_words.append(contractions.fix(word))  
    
  expanded_text = ' '.join(expanded_words)
  return expanded_text

In [ ]:
def clean_text(text):
    cleaned_text = text.lower()
    # remove emojis
    cleaned_text = re.sub(r'[&#?[\d|\w]+;', '', cleaned_text)
    # remove hyperlinks    
    cleaned_text = re.sub(r'((http|https)://)?[a-zA-Z0-9./?:@-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9.&/?:@\-_=#])*', '', cleaned_text)
    # substitute punctuactions by blank spaces
    cleaned_text = re.sub(r'[^\w\s_]+|_', ' ', cleaned_text)
    cleaned_text = cleaned_text.replace("_", "")
    # remove numbers
    cleaned_text = re.sub(r'\d+\w+|\d+', '', cleaned_text)
    #cleaned_text = re.sub(r'_', ' ', cleaned_text)
    # delete extra spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)   
    return cleaned_text

In [ ]:
heading_cleaned = []
for t in df['heading']:
  heading_cleaned.append(clean_text(t))
df['heading_cleaned'] = heading_cleaned  

In [ ]:
df[['heading', 'heading_cleaned']].head(20)

,heading,heading_cleaned
0,Madden NFL 25 XBOX 360. Brand New!,madden nfl xbox brand new
1,looking for room to rent.,looking for room to rent
2,two DS game,two ds game
3,map,map
4,Good Looking Asian Sensation N aughty Girl ---...,good looking asian sensation n aughty girl ind...
5,map,map
6,"Panasonic ,2doors fridge(238L)($220 with deliv...",panasonic fridge with delivery warranty
7,SWEET FRIENDLY COMPANION:- monaliza,sweet friendly companion monaliza
8,&#10031; Indulge yourself to the body to body ...,indulge yourself to the body to body sensual ...
9,Enough,enough


In [ ]:
# df.to_excel('/content/gdrive/MyDrive/ColabNotebooks/PostClassifier/cleaned_data.xlsx', index=False)

Calculamos una nueva columna sintética con la información de las columnas perdictoras.

In [ ]:
df['header_tranformed']  = df['city'] + ' ' + df['section'] + ' '+ df['heading_cleaned']

## Entrenamiento del modelo

¿Cuáles son los resultados óptimos que podemos esperar de nuestro modelo, dados los datos de entrada?

La clasificación de texto es una tarea que a los humanos se nos da especialmente bien, por lo que, si conseguimos que el modelo ahaga unas predicciones con una precisión similar a la de un humano, podemos estar satisfechos con el resultado.

Para establecer un nivel de referencia, tomaremos una muestra y la clasificaremos manualmente.

Tomamos 100 datos de muestra al azar:

In [ ]:
data_sample = df[['city', 'section', 'heading', 'category']].sample(frac=1).head(100)

Persistimos la muestra en un fichero externo.

In [ ]:
#data_sample.to_excel('/content/gdrive/MyDrive/ColabNotebooks/PostClassifier/sample_complete.xlsx', index=False)

Tras hacer la clasificación automática, se han observado varios problemas:

*   Existen muchos registros con texto aleatorio como 'map', que son imposibles de clasificar
*   Otros registros tienen una clasificación ambigüa o directamente están mal clasificados

Realizando la clasificación manual, se ha obtenido un 75% de acierto. Añadimos, un tanto arbitrariamente, un margen de un 5% de posibles errores cometidos en el proceso manual, más posibles sesgos introducidos en el método de muestreo, por ejemplo, haber tomado por casualidad textos especialmente difíciles de clasificar.

Por tanto, tomamos un 75-80% como uvalor de referencia a alcanzar. En ningún caso s eespera, con la calidad de entrada, una precisión alta del modelo en la clasificación.



¿Existen clases con pocos datos?

In [ ]:
df.groupby(['category'])['category'].count()

category
activities             639
appliances            2496
artists                793
automotive            2040
cell-phones           3066
childcare             2218
general               1760
household-services    2424
housing                688
photography           2427
real-estate           1806
shared                4074
temporary             2693
therapeutic           3808
video-games           2214
wanted-housing        2441
Name: category, dtype: int64

### Arquitectura del modelo

Tokenización del vocabulario:

In [ ]:
X = df.loc[:, df.columns != 'category']
y = df['category']
X_train, X_, y_train, y_ = train_test_split(X, y,test_size=0.40, random_state=1, shuffle=True)
X_cv, X_test, y_cv, y_test = train_test_split(X_,y_,test_size=0.50, random_state=1)

In [ ]:
def one_hot(array):
    unique, inverse = np.unique(array, return_inverse=True)
    onehot = np.eye(unique.shape[0])[inverse]
    return onehot


In [ ]:
def tokenize_corpus(corpus):
  my_tokenizer = Tokenizer(oov_token="<OOV>")
  my_tokenizer.fit_on_texts(corpus)
  return my_tokenizer

#my_tokenizer.texts_to_sequences(corpus)

In [ ]:
tokenized_corpus = tokenize_corpus(df['header_tranformed'])

In [ ]:
len(tokenized_corpus.word_index)

13000

Dado que los textos no son excesivamente largos, no parece que tengamos que hacer frente al problema del desvanecimiento del gradiente, por lo que usaremos un modelo LSTM con una base de word embedding preentrenada.

In [ ]:
#!unzip gdrive/MyDrive/ColabNotebooks/PostClassifier/glove*.zip

In [ ]:
def create_embedding_layer(tokenized_corpus, glove_dir, embedding_dim, trainable, max_len_sequence):
  # Calculate the matrix weights for the input corpus
  #params:
  # tokenizedd_corpus: the text after the cleaning and tokenization process
  # glove_dir: the method uses the glove pretrained embedding vector space. The value of this param is the directory where it is allocated
  # The dimension of the embedding
  
  embeddings_index = {}

  f = open(glove_dir)
  for line in f:
      values = line.split()
      word = values[0]
      coefs = np.asarray(values[1:],dtype='float32')
      embeddings_index[word] = coefs
  f.close()

  print('Found %s word vectors.' % len(embeddings_index))

  vocab_size = len(tokenized_corpus.word_index) + 1

  embedding_matrix = np.zeros((vocab_size,embedding_dim))
  for word , i in tokenized_corpus.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

  embedding_layer = Embedding(input_dim=len(tokenized_corpus.word_index)+ 1,
                              output_dim=embedding_dim,
                              weights=[embedding_matrix],
                              input_length=max_len_sequence,
                              trainable=trainable)        
  return embedding_layer

In [ ]:
#glove_dir = '/content/gdrive/MyDrive/ColabNotebooks/PostClassifier/merged_data.csv/glove.6B.100d.txt'
embedding_layer = create_embedding_layer(tokenized_corpus, glove_dir = "glove.6B.100d.txt", embedding_dim = 100, trainable = False, max_len_sequence = 80)

Establecemos la capa de embedding del modelo

In [ ]:
def build_model(embedding_layer, embedding_dim, max_len_sequence, num_classes):
  model=Sequential()
  model.add(embedding_layer)
  model.add(LSTM(units=64, input_shape=(max_len_sequence, embedding_dim)))
  model.add(Dropout(0.7))
  model.add(Dense(units=num_classes, activation='softmax'))

# kernel_regularizer=tf.keras.regularizers.l2(0.1)
  return model

In [ ]:
model = build_model(embedding_layer, embedding_dim = 100, max_len_sequence = 80, num_classes = 16)

In [ ]:
model.summary()

Compilamos el modelo.

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.01), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Construímos  los datos de entrenamiento, validación y test

In [ ]:
#sequences = tokenized_corpus.texts_to_sequences(df['heading_cleaned'])
X_train_seq = pad_sequences(tokenized_corpus.texts_to_sequences(X_train['header_tranformed']), maxlen=80)
y_train_seq = one_hot(y_train)

X_cv_seq = pad_sequences(tokenized_corpus.texts_to_sequences(X_cv['header_tranformed']), maxlen=80)
y_cv_seq = one_hot(y_cv)

X_test_seq = pad_sequences(tokenized_corpus.texts_to_sequences(X_test['header_tranformed']), maxlen=80)
y_test_seq = one_hot(y_test)

In [ ]:
# fit the model
history = model.fit(X_train_seq, y_train_seq,epochs=20, batch_size=100, verbose=1)

In [ ]:
val_loss, val_acc = model.evaluate(X_cv_seq, y_cv_seq)
print()
print("Test accuracy = ", val_acc)

223/223 [==============================] - 4s 16ms/step - loss: 0.4402 - accuracy: 0.8259

Test accuracy =  0.8259097933769226


In [ ]:
test_loss, test_acc = model.evaluate(X_test_seq, y_test_seq)
print()
print("Test accuracy = ", test_acc)

223/223 [==============================] - 4s 18ms/step - loss: 0.4452 - accuracy: 0.8257

Test accuracy =  0.8256532549858093


In [ ]:
#  = np.argmax(model.predict(X_test_seq), axis=1) X_test['pred']

223/223 [==============================] - 3s 15ms/step


In [ ]:
X_test['real'] = y_test

In [ ]:
preds = np.argmax(model.predict(X_test_seq), axis=1) 
verbose_pred = []
for index in preds:
  verbose_pred.append(sorted(y_test.unique())[index])

223/223 [==============================] - 5s 21ms/step


In [ ]:
X_test['pred'] = verbose_pred

Se ha conseguido una accuracy en torno a lo que puede conseguir un humano, por lo que se considera resuelto el problema.

# Análisis de errores

Como última fase del proyecto, analizamos manualmente algunos errores para ver qué patrón presentan.

In [ ]:
X_test[X_test['pred'] != X_test['real']]

,city,section,heading,heading_cleaned,header_tranformed,pred,real
24441,seattle,services,Buyer Inspection we check everything on any ma...,buyer inspection we check everything on any make,seattle services buyer inspection we check eve...,real-estate,automotive
24025,seattle,community,8 Great Parenting Tips for Sarcastic Parents,great parenting tips for sarcastic parents,seattle community great parenting tips for sa...,childcare,general
27563,chicago,housing,single dwelling living space&gt;READ,single dwelling living read,chicago housing single dwelling living read,shared,wanted-housing
26681,chicago,housing,map,map,chicago housing map,temporary,shared
2952,chicago,for-sale,map,map,chicago for-sale map,appliances,cell-phones
...,...,...,...,...,...,...,...
9109,chicago,for-sale,Sankyo Dualux 1000 Dual Projector Super 8,sankyo dualux dual projector super,chicago for-sale sankyo dualux dual projector ...,appliances,photography
22416,seattle,for-sale,map,map,seattle for-sale map,appliances,video-games
2529,newyork,housing,Furnished gigantic room in 3BR UWS Apt!,furnished gigantic room in uws apt,newyork housing furnished gigantic room in uws...,shared,temporary
14805,newyork,housing,map,map,newyork housing map,temporary,shared
